# Improved GPT

このファイルではGPT from scratchで学習したモデルの改善を目標とする。このファイルの立ち位置は同リポジトリに含まれている
- pytorch_command.ipynb
- attention_from_scratch.ipynb
- GPT_from_scratch.ipynb<br>
の次に読むことを想定されている。

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
import warnings
warnings.simplefilter('ignore')
print("CUDA環境が壊れていないことを祈りながら確認-> ", torch.cuda.is_available())
import subprocess
from time import time, sleep

CUDA環境が壊れていないことを祈りながら確認->  True


### PreLN
GPT from scratchでは、学習中に勾配が変化しない状態がいつまで経っても続く時があった。<br>
これはLayer Normalization層がMultiheadAttentionやFFN層の後に置かれるというPostLNの形をとっているからであると最近では言われている<br>
このため、改善点の一つとして、サブレイヤーの前にLayer Normalization層を置くPreLNを行う。<br>
最近のGPTモデルではPreLNを行なっているようである。

In [2]:
class PreLNGPTDecoderLayer(nn.Module):
    def __init__(self, embedding_dim, ffn_dim, num_heads, drop_out_rate = 0., layer_eps=1e-05, batch_first = False):
        super().__init__()
        self.maskedmultiheadattention = nn.MultiheadAttention(embedding_dim, num_heads,batch_first=batch_first)
        self.dropout_selfattn = nn.Dropout(p = drop_out_rate)
        self.layernorm_selfattn = nn.LayerNorm(embedding_dim, eps = layer_eps)

        self.ffn = nn.Sequential(nn.Linear(embedding_dim, ffn_dim), nn.GELU(), nn.Linear(ffn_dim, embedding_dim))#GELUに変更
        self.layernorm_ffn = nn.LayerNorm(embedding_dim, eps = layer_eps)
        self.dropout_ffn = nn.Dropout(p = drop_out_rate)

    def forward(self, x, pad_mask_self = None, mask_self=None):
        #PreLNにする
        dx = self.layernorm_selfattn(x)

        dx, _ = self.maskedmultiheadattention(dx,dx,dx,key_padding_mask = pad_mask_self, attn_mask = mask_self)

        dx = self.dropout_selfattn(dx)

        x = x+dx

        dx = self.layernorm_ffn(x)

        dx = self.dropout_ffn(self.ffn(dx))

        x = x + dx
        return x

### 改善したGPT

このPreLNGPTDecoderLayerを用いてGPTモデルを制作する。ところで、今回はGPTのgenerate_sentence関数でもtemperatureとtopKという手法を追加した。<br>
これにより多様性のある文章が生成される。

Temperature: Tはトークンの予測確率$p_{i}$を以下のように変換するパラメーターである。<br>
$$
p_{i} \to \dfrac{\exp(\dfrac{p_i}{T})}{\sum_{j}\exp(\dfrac{p_j}{T})}
$$
Tを大きくすればするほどもともとの確率が低いトークンの確率が高くなる。<br>
topKは予測確率の一番高いトークンを選ぶのではなく、予測確率の高いトークンのうち、上位K個を出力対象としてランダムに選ぶ手法である。<br>
これらの手法を組み合わせるにより、出力は毎回ランダムであるが、多様性のある文章が生成されるようになる。

In [3]:
class GPT(nn.Module):
    def __init__(self, vocab_size, embedding_dim, ffn_dim, num_heads, drop_out_rate = 0.,\
                  layer_eps=1e-05, batch_first = False, T = 10000, N = 1):
        super().__init__()
        #Tはmax_lenを表している
        self.embedding = nn.Embedding(vocab_size, embedding_dim,)
        self.positional_embedding = nn.Embedding(T, embedding_dim)
        self.decoder = nn.ModuleList([PreLNGPTDecoderLayer(embedding_dim, ffn_dim, num_heads, drop_out_rate,\
                                                               layer_eps, batch_first) for _ in range(N)])
        self.linear = nn.Linear(embedding_dim, vocab_size, bias = False)
        self.vocab_size = vocab_size
    def forward(self, x, y = None,pad_mask_self = None, mask_self=None):
        """
        yはxを1つだけずらしたデータである
        x = data[a:b]なら、y = data[a+1:b+1]となる。
        """
        x = self.embedding(x)
        pos = torch.arange(0,x.size(1),dtype=torch.long).unsqueeze(0).to(x.device)
        pos = self.positional_embedding(pos)
        x = x + pos
        for layer in self.decoder:
            x = layer(x, pad_mask_self = pad_mask_self, mask_self = mask_self)
        x = self.linear(x)
        if y != None:
            loss = F.cross_entropy(x.view(-1, x.size(-1)), y.view(-1), ignore_index=-1) 
            #ignore_index=-1はyをonehotベクトル化しないでcross_entropyを使うために使用
            pred = x.argmax(dim = -1).detach().cpu()
            return loss,pred
        loss = None
        pred = x[:,[-1],:]
        return loss, pred
    def create_mask(self, x: torch.tensor, x_pad: int, device: str):
        """
        (batch_size, sequence_length, embedding_dim)の入力を想定
        """
        """
        Trueが無視される値であることに注意すること
        """
        seq_len = x.size(1)
        #srcのマスク制作
        padding_mask = (x == x_pad)
        mask = torch.triu(torch.ones(size = (seq_len, seq_len))==1).transpose(0,1) #下三角行列を作る
        mask = mask.float().masked_fill(mask == 0, float("-inf")).masked_fill(mask==1.,float(0.0)).to(device)
        return padding_mask, mask

    @torch.no_grad()
    def generate(self,bos: str, sentence_size, tokenizer, device):
        self.eval()
        bos_tokenized = tokenizer.encode_ordinary(bos)
        bos_tokenized = bos_tokenized[-sentence_size:]
        bos_tokenized = torch.LongTensor([bos_tokenized])
        _, add_sentence = self(bos_tokenized.to(device))
        self.train()
        return add_sentence
    
    @torch.no_grad()
    def generate_sentence(self, bos: str, sentence_size, generate_tokens, tokenizer, device, top_K = None, temperature = 1.0):
        return_sentence = bos
        for i in range(generate_tokens):
            add_sentence = self.generate(return_sentence, sentence_size, tokenizer,device)
            add_sentence = add_sentence[:,-1,:] / temperature #(1, vocab_size)
            if top_K is not None:
                v, _ = torch.topk(add_sentence, min(top_K, add_sentence.size(-1)))
                #v[:, [-1]]がtopkの中でも最小値を取る。これより小さいやつは予想に含めない。
                add_sentence[add_sentence < v[:, [-1]]] = -float('Inf')
            probs = F.softmax(add_sentence, dim = -1)
            idx_next = torch.multinomial(probs, num_samples=1)
            return_sentence += tokenizer.decode_batch(idx_next.tolist())[0]
        return return_sentence

それでは訓練に移ろう、使うデータはGPT＿from＿scratchでも製作したデータである。

In [4]:
train_data = np.memmap("bin/train.bin", dtype = np.uint16, mode = "r")
val_data = np.memmap("bin/val.bin", dtype = np.uint16, mode = "r")

In [17]:
sentence_size = 1024
batch_size = 6
device = "cuda" if torch.cuda.is_available() else "cpu"
def get_batch(split: str, batch_size = 256,device = "cpu")->torch.Tensor:
    data = train_data if split == 'train' else val_data
    index = torch.randint(len(data) - sentence_size, (batch_size,))
    x = torch.stack([torch.from_numpy((data[i:i+sentence_size]).astype(np.int64)) for i in index])
    y = torch.stack([torch.from_numpy((data[i+1:i+1+sentence_size]).astype(np.int64)) for i in index])
    if device == "cuda":
        return x.pin_memory().to(device, non_blocking=True), y.pin_memory().to(device, non_blocking=True)
    return x.to(device), y.to(device)

In [18]:
import tiktoken
device = "cuda" if torch.cuda.is_available() else "cpu"
embedding_size = 768
num_heads = 6
tokenizer = tiktoken.get_encoding("gpt2")
#KarpathyのminGPTを参考に、パラメーターを設定した。
depth = 6 
gpt = GPT(50257, embedding_size, embedding_size*4, num_heads, 0, batch_first=True, T = sentence_size, N = depth).to(device) 
#事前学習のときはDropout無し、ファインチューニングのときはありが好ましい
warmup_iters = 2000

optimizer = torch.optim.Adam(gpt.parameters(), lr = 0.0001)

cos scheduler関数は少し変更を加え、clipメソッドを用いてmax_lrとmin_lrの間で必ず学習率が出力されるようにした。

In [19]:
max_lr = 2.5e-5
min_lr = 2.5e-6
max_iters = 10000
def get_lr(cur_iter):
    #cur_iter現在のiteration
    if cur_iter < warmup_iters:
        return max_lr * cur_iter / warmup_iters
    return (max_lr * (np.cos(cur_iter / max_iters * np.pi) + 1)).clip(min_lr, max_lr)

学習の際もただパラメーターを保管するだけではなく、パラメーター、最も良い検証データの損失、その時のiterationを記録するようにした。<br>
LLMは学習中に突然損失が大きくなるときがあるので、損失がNaNとなったときは学習を停止し、その時のパラメーターは保存しないようにした。<br>
こうしないとパラメーターの一部が知らぬ間にNaNとなっている場合がある。

In [11]:
"""最初の訓練時のコードでは以下のように初期化を行なう。
import gc
from tqdm import tqdm
batch_iteration = 128
scaler = torch.cuda.amp.GradScaler(enabled=True)
best_loss = 1e9
begin = 0
val_iteration = 1
"""

'最初の訓練時のコードでは以下のように初期化を行なう。\nimport gc\nfrom tqdm import tqdm\nbatch_iteration = 128\nscaler = torch.cuda.amp.GradScaler(enabled=True)\nbest_loss = 1e9\nbegin = 0\nval_iteration = 1\n'

In [ ]:
import gc
from tqdm import tqdm
checkpoint = torch.load("best_checkpoint.bin", map_location="cpu") #チェックポイントがあるなら使う
batch_iteration = 256
scaler = torch.cuda.amp.GradScaler(enabled=True) 
best_loss = checkpoint["best_loss"]
begin = checkpoint["iter"]
val_iteration = 10
gpt.load_state_dict(checkpoint["model"])
optimizer.load_state_dict(checkpoint["optimizer"])
del checkpoint
gc.collect()
torch.cuda.empty_cache()
sleep(5)
gpt.train()
for cur_iter in tqdm(range(begin,max_iters)):
    optimizer.lr = get_lr(cur_iter+1)
    for batch_iter in range(batch_iteration):
        optimizer.zero_grad()
        with torch.amp.autocast(device_type=device, dtype=torch.bfloat16):
            x,y = get_batch("train",batch_size=batch_size,device=device)
            padding_mask, mask = gpt.create_mask(x, 0, device)
            loss, pred = gpt(x,y,padding_mask,mask)
        scaler.scale(loss).backward() 
        scaler.step(optimizer) 
        scaler.update()
        del x, y
        del padding_mask, mask
        del loss
        del pred
        gc.collect()
        torch.cuda.empty_cache()
    valid_loss = 0
    for val_iter in range(val_iteration):
        with torch.no_grad(): #こうしないとCUDAERRORが起きる
            with torch.amp.autocast(device_type=device, dtype=torch.bfloat16):
                x,y = get_batch("valid",batch_size=batch_size,device=device)
                padding_mask, mask = gpt.create_mask(x, 0, device)
                loss, pred = gpt(x,y,padding_mask,mask)
                valid_loss += loss.detach()
                del loss
                del x, y
                del padding_mask, mask
                del pred
                gc.collect()
                torch.cuda.empty_cache()
    if best_loss > valid_loss.item() / val_iteration:
        best_loss = valid_loss.item() / val_iteration
        checkpoint = {
            "model": gpt.state_dict(),
            "optimizer": optimizer.state_dict(),
            "scaler": scaler,
            "iter": cur_iter,
            "best_loss": best_loss,
        }
        torch.save(checkpoint, "best_checkpoint.bin")
        print("params updated. BestLoss: ", best_loss)
        print("Val all loss", valid_loss.item())
        with open("learning_detail.txt","w") as f:
            f.write("学習状況\n")
            f.write(f"iter: {cur_iter}\n")
            f.write(f"hyper params: \n")
            f.write(f"vocab_size: 50257, embedding size: {embedding_size}, ffn: {embedding_size*4}, num_heads: {num_heads}, Depth: {depth}, sentnce_size: {sentence_size}\n")
            f.write(f"lr: {optimizer.lr},best_loss: {best_loss}\n")
            f.close()
    if torch.isnan(valid_loss):
        print("Loss is NaN!")
        break
    checkpoint = {
    "model": gpt.state_dict(),
    "optimizer": optimizer.state_dict(),
    "scaler": scaler,
    "iter": cur_iter,
    "best_loss": best_loss,
    "loss": valid_loss.item()
    }
    torch.save(checkpoint, "latest_checkpoint.bin")
    with open("learning_detail_latest.txt","w") as f:
        f.write("学習状況\n")
        f.write(f"iter: {cur_iter}\n")
        f.write(f"hyper params: \n")
        f.write(f"vocab_size: 50257, embedding size: {embedding_size}, ffn: {embedding_size*4}, num_heads: {num_heads}, Depth: {depth}, sentnce_size: {sentence_size}\n")
        f.write(f"lr: {optimizer.lr},best_loss: {best_loss}\n")
        f.write(f"val_loss: {valid_loss.item() / val_iteration}\n")
        f.close()
        del valid_loss
        gc.collect()
        torch.cuda.empty_cache()

このコードを何日間かの間、リポジトリ製作者のパソコンで実行した結果のなかで、<br>
最もマシと考えられる結果が得られたときのパラメーターを用いて出力を見てみることにする。<br>
このチェックポイントファイルは1.4GB以上あるのでGithub上にあげることができないのはお許しください。<br>
このときの学習状況は以下の通りとなった。<br>
学習状況<br>
iter: 325/10000<br>
hyper params: <br>
vocab_size: 50257, embedding size: 768, ffn: 3072, num_heads: 6, Depth: 6, sentnce_size: 1024<br>
lr: 4.075000000000001e-06, best_loss: 3.849387884140015<br>

In [87]:
checkpoint = torch.load("best_checkpoint.bin", map_location="cpu")

In [88]:
gpt.load_state_dict(checkpoint["model"])

<All keys matched successfully>

In [108]:
print(gpt.generate_sentence("He has two sisters, and ", \
                            sentence_size, 128, tokenizer,device,top_K=20,temperature = 2))

He has two sisters, and ents, but not a great-uncle, The Boy in all that is the same type-snow, both, but a good name. I know there's two very bad and they are all but a good reason they've probably got good in comparison, in fact I know that they haven't the only the most of these guys in our division of the first names.

'The same old guys; I'm not the opposite that this is that, either (although they just got better guys are that, in that team. You can probably have more of their name: they probably just the same types as those other names and


In [109]:
print(gpt.generate_sentence("I always have a breakfast.", \
                            sentence_size, 128, tokenizer,device,top_K=20,temperature = 2))

I always have a breakfast. She just isn�t going to go in the bed. If someone wants a couple or don�t even care to make it to go into hospital the way, then some kid to sleep. I am sick, but we are a good time for my mom and they're so happy that we can�t need a little bit sleep, she�s so you know she needs. The fact that it�t. I have a big sleep to keep me sleep well so that a good day, the baby that she loves a better job, the only way to get around, they�t. My mom.<|endoftext|> is a sleep out


In [110]:
print(gpt.generate_sentence("Japan is a country, which", \
                            sentence_size, 128, tokenizer,device,top_K=20,temperature = 2))

Japan is a country, which has the longest-populifying population of about 10 people, but most recently. But this trend has seen a trend in the global poverty. A drop by population of 10 is emerging growth since 2010. What are people have seen some of the trend that is now is rising across all the global food, but more of which has seen some is experiencing growth trends in recent years, and growing. For most is a percentage since then growth since 2010. Now that growth since 2006 and growing by growth means increasing growth over the over half is growing more than this increase is just one, in part, with over 10, a trend in 2007 is


GPT_from_scratch.ipynbよりかはまともな出力が得られたと考えられる。実際にPreLNに変えたことで学習はいくらか向上したようである。<br>
今回製作したGPTは最も初代のGPT1であるため、タスクに適合させるためには、<br>
転移学習を行なう必要がある(いわゆるZero shot，Few shotを意図して作られてはいない)(詳しくは[https://data-analytics.fun/2020/04/18/understanding-openai-gpt/](https://data-analytics.fun/2020/04/18/understanding-openai-gpt/)などが詳しい)<br>

それでも制作に当たり反省点がいくらかある。それを述べていこう。<br>
・10000 epoch学習させるつもりだったが、325iterでの結果が最も良かった。<br><br>
これは学習のバッチサイズが6と非常に小さく、一回のパラメーター更新で6個の文章を用いてしか<br>
パラメーターを更新することができなかったことが原因と思われる<br>
実は、学習が完了してから勾配累積という方法を知り、それを用いて擬似的にバッチサイズを増やすことができることを学んだ。<br>
次に大規模な学習を行なうときはこの手法を最初から用いたほうが良い。

・モデルのサイズを小さくしてしまった。<br><br>
モデルのサイズがGPUの関係で小さくなってしまった。次やるときは社会人になって80GBぐらいのGPU買ってやるしかない。<br>

・さらに自然な出力に向けて<br><br>

GPT2, GPT3のようにタスクをこなせるようにFew shotの実現ができるようなデータでも学習を行いたい。<br>
Reinfrce Learning from Human Feedbackのようにまだまだ改善できる手法は残っているのでこれらの実装を行っていきたい<br>

以上でこのAttention_from_scratchリポジトリで提供する全コンテンツの解説を終了したいと思います。<br>
はじめはAttentionを1から制作するリポジトリを作るという意気込みで始めましたが、最近の深層自然言語処理をまとめたようなリポジトリになってしまったなと思います。<br>

もしこのリポジトリが役に立つ人がいらっしゃいましたら非常に光栄です。<br>